In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow as tf
import sklearn.model_selection as skl

print(tf.__version__)

2.6.0


In [2]:
import glob
import re
import os

txt_file_path = './lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당
print(txt_list)
raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='utf-8') as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

['./lyrics/Lil_Wayne.txt', './lyrics/radiohead.txt', './lyrics/missy-elliott.txt', './lyrics/rihanna.txt', './lyrics/Kanye_West.txt', './lyrics/cake.txt', './lyrics/amy-winehouse.txt', './lyrics/nirvana.txt', './lyrics/nicki-minaj.txt', './lyrics/adele.txt', './lyrics/jimi-hendrix.txt', './lyrics/kanye-west.txt', './lyrics/r-kelly.txt', './lyrics/paul-simon.txt', './lyrics/lil-wayne.txt', './lyrics/janisjoplin.txt', './lyrics/lorde.txt', './lyrics/disney.txt', './lyrics/britney-spears.txt', './lyrics/alicia-keys.txt', './lyrics/dr-seuss.txt', './lyrics/nickelback.txt', './lyrics/bob-marley.txt', './lyrics/bruce-springsteen.txt', './lyrics/kanye.txt', './lyrics/drake.txt', './lyrics/nursery_rhymes.txt', './lyrics/bieber.txt', './lyrics/bob-dylan.txt', './lyrics/lin-manuel-miranda.txt', './lyrics/dolly-parton.txt', './lyrics/notorious_big.txt', './lyrics/bjork.txt', './lyrics/dj-khaled.txt', './lyrics/notorious-big.txt', './lyrics/blink-182.txt', './lyrics/lady-gaga.txt', './lyrics/beatl

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
raw_preprocess = []

for raw in raw_corpus:
    space_split = raw.split(' ')
    if(len(space_split) >= 15):
        raw_corpus.remove(raw)
    else:
        sentence = preprocess_sentence(raw)
        #print(sentence)
        raw_preprocess.append(sentence)

print(raw_preprocess[0])
print("정제 후 데이터 크기:", len(raw_preprocess))


<start> they call me mr carter i kissed the daughter <end>
정제 후 데이터 크기: 169291


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> they call me mr carter i kissed the daughter <end>',
 '<start> of the deads forehead i killed the father <end>',
 '<start> spilled the heart of a mildew hater <end>',
 '<start> i will put them body on chill like glaciers <end>',
 '<start> gracias im crazy yes its obvious <end>',
 '<start> going against me is atheist <end>',
 '<start> i got my angels on my shoulders and a quarter of that angel dust <end>',
 '<start> i aint sniffin im just pitchin ya honor i aint snitchin ya honor <end>',
 '<start> hate bitch nixgas bitches with power <end>',
 '<start> vacate when the kitchen get hotter <end>']

In [6]:
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [7]:
tensor, tokenizer = tokenize(corpus)
print(len(tensor))
print(tensor[:3, :10])

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
#2 : start
#3 : end

[[   2   43  154 ...    0    0    0]
 [   2   20    6 ...    0    0    0]
 [   2 4760    6 ...    0    0    0]
 ...
 [   2    5   24 ...    0    0    0]
 [   2    5   24 ...    0    0    0]
 [   2    5   61 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4849d26f70>
166855
[[    2    43   154    12   587  2439     5  1424     6   959]
 [    2    20     6 13045  3751     5  1007     6   630     3]
 [    2  4760     6   139    20     9 10838  2021     3     0]]
1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t
16 : s
17 : that
18 : on
19 : .
20 : of
21 : your
22 : m
23 : we
24 : like
25 : all
26 : is
27 : be
28 : for
29 : up
30 : with
31 : so
32 : can
33 : but
34 : know
35 : just
36 : don
37 : love
38 : no
39 : what
40 : got
41 : oh
42 : this
43 : they
44 : ?
45 : get
46 : when
47 : she
48 : do
49 : yeah
50 : now
51 : baby
52 : if
53 : re
54 : go
55 : he
56 : out
57 : was
58 : !
59 : down
60 : o

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   43  154   12  587 2439    5 1424    6  959    3    0    0    0
    0    0    0    0    0]
[  43  154   12  587 2439    5 1424    6  959    3    0    0    0    0
    0    0    0    0    0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 15001), dtype=float32, numpy=
array([[[ 1.42213694e-05, -1.27976367e-04,  3.79723424e-05, ...,
         -2.82274617e-04,  1.31516441e-04,  1.27560261e-05],
        [ 3.15234752e-06,  7.84541589e-06,  7.20050302e-05, ...,
         -3.76530661e-04,  2.49884761e-04,  8.66913251e-05],
        [ 9.70915498e-05,  6.23040032e-05,  5.88929652e-05, ...,
         -4.12116729e-04,  1.30069733e-04,  1.01931466e-04],
        ...,
        [ 5.30376332e-04, -1.74376764e-03,  1.12361822e-03, ...,
          5.52395999e-04,  4.10958339e-04, -1.08186947e-03],
        [ 6.30855153e-04, -2.02226406e-03,  1.11965521e-03, ...,
          6.51987910e-04,  5.61296823e-04, -1.24626444e-03],
        [ 7.18693889e-04, -2.26605544e-03,  1.08971400e-03, ...,
          7.43431272e-04,  7.02517980e-04, -1.40494574e-03]],

       [[ 1.42213694e-05, -1.27976367e-04,  3.79723424e-05, ...,
         -2.82274617e-04,  1.31516441e-04,  1.27560261e-05],
        [-2.64290757e-05, -1.73077584e-04,  5

In [12]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=30) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/30
651/651 [==============================] - 173s 258ms/step - loss: 2.8005
Epoch 2/30
651/651 [==============================] - 177s 272ms/step - loss: 2.3909
Epoch 3/30
651/651 [==============================] - 178s 273ms/step - loss: 2.2440
Epoch 4/30
651/651 [==============================] - 178s 273ms/step - loss: 2.1362
Epoch 5/30
651/651 [==============================] - 178s 273ms/step - loss: 2.0440
Epoch 6/30
651/651 [==============================] - 178s 273ms/step - loss: 1.9591
Epoch 7/30
651/651 [==============================] - 178s 273ms/step - loss: 1.8790
Epoch 8/30
651/651 [==============================] - 178s 274ms/step - loss: 1.8043
Epoch 9/30
651/651 [==============================] - 178s 274ms/step - loss: 1.7345
Epoch 10/30
651/651 [==============================] - 178s 274ms/step - loss: 1.6692
Epoch 11/30
651/651 [==============================] - 179s 274ms/step - loss: 1.6088
Epoch 12/30
651/651 [==============================] - 179s 274

In [14]:
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.

In [16]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [29]:
generate_text(model, tokenizer, init_sentence="<start> i love ")
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '